In [ ]:
getEmpiricalDistrRaw <- function(n, k, reps=1e7){
    # Just generates the samples and leaves them in matrix form,
    # one per row, unsorted.
    samReplicates <- t(replicate(reps, sort(sample(n, k))))
    return(samReplicates)
}
                                         
getEmpiricalDistr <- function(samReplicates){
    # Takes input from getEmpiricalDistrRaw
    uniqueSampleVec <- unique(samReplicates)
    uniqueSamples <- vector("list", nrow(uniqueSampleVec))
    for(i in seq_along(uniqueSamples)){
        sam <- uniqueSampleVec[i, ]
        uniqueSamples[[i]]$sample <- sam
        uniqueSamples[[i]]$freq <- sum(apply(samReplicates, 1, function(row) all(row==sam)))
    }
    return(uniqueSamples)
}
                                             
                                             
getItemCounts <- function(samReplicates){
  # Takes input from getEmpiricalDistrRaw
  itemCounts <- data.frame(table(samReplicates))
  colnames(itemCounts) <- c("Item", "Count")
  return(itemCounts)
}                                         

                                             
getEmpiricalDistr_old <- function(n, k, reps=1e7){
  samReplicates <- t(replicate(reps, sort(sample(n, k))))
  uniqueSampleVec <- unique(samReplicates)
  uniqueSamples <- vector("list", nrow(uniqueSampleVec))
  for(i in seq_along(uniqueSamples)){
    sam <- uniqueSampleVec[i, ]
    uniqueSamples[[i]]$sample <- sam
    uniqueSamples[[i]]$freq <- sum(apply(samReplicates, 1, function(row) all(row==sam)))
  }
  return(uniqueSamples)
}
                                                                          

getItemCounts_old <- function(n, k, reps = 10^7){
  samReplicates <- t(replicate(reps, sort(sample(n, k))))
  itemCounts <- data.frame(table(samReplicates))
  colnames(itemCounts) <- c("Item", "Count")
  return(itemCounts)
}

                                         
getItemFreq <- function(itemCounts, reps = 10^7){
  itemCounts$Probability <- itemCounts$Count/reps
  return(itemCounts)
}


computeMaxProbRatio <- function(probs){
  return(max(probs) / min(probs))
}

                                         
conductChisqTest <- function(counts){
  res <- chisq.test(counts)
  return(list(
    "Statistic" = res$statistic,
    "DF" = res$parameter,
    "Pvalue" = res$p.value
  ))
}

In [ ]:
library(testthat)

distrNormalRange <- function(w, n){
  # CDF of the range of n IID standard normals, evaluated at w
  tmp <- integrate(function(x) dnorm(x)*(pnorm(x+w)-pnorm(x))^(n-1), lower = -Inf, upper = Inf)
  n*tmp$value
}


test_distrNormalRange <- function(){
  n = 100
  set.seed(12345)
  
  # Works!
  empiricalRangeDistr <- replicate(100000, {
    tmp <- rnorm(n)
    max(tmp) - min(tmp)
  })
  for(w in seq(3, 6, by = 0.5)){
    emp <- mean(empiricalRangeDistr <= w)
    expect_equal(distrNormalRange(w, n), emp, tolerance = 0.005)
  }
}

distrMultinomialRange <- function(w, n, k){
  # CDF of the range of multinomial variables evaluated at w
  # n draws, k categories each having probability 1/k
  cutoff <- (w - 1/(2*n))*sqrt(k/n)
  return(distrNormalRange(cutoff, k))
}

test_distrMultinomialRange <- function(){
  reps = 10000
  
  bins = 15
  set.seed(12345)
  
  # Works!
  empiricalRangeDistr <- replicate(100000, {
    tmp <- rmultinom(n = 1, size = reps, prob = rep(1/bins, bins))
    diff(range(tmp))
  })
  for(w in (1:20)*10){
    emp <- mean(empiricalRangeDistr <= w)
    expect_equal(distrMultinomialRange(w, reps, bins), emp, tolerance = 0.015)
  }
}
                   
# Will be silent if there are no errors
test_distrNormalRange()
test_distrMultinomialRange()

In [ ]:
# Boilerplate stuff

reps <- 10^7
n <- c(13, 30, 90)
k <- c(4, 10, 20)

maxProb <- c()
minProb <- c()
meanProb <- c()
maxProbRatio <- c()
nvalues <- c()
kvalues <- c()
prng <- c()
seed <- c()

# FO = first order selection probabilities
chisqStatistic_FO <- c()
chisqDF_FO <- c()
chisqPvalue_FO <- c()
rangeStat_FO <- c()
rangePvalue_FO <- c()

# US = unique sample selection probabilities
chisqStatistic_US <- c()
chisqDF_US <- c()
chisqPvalue_US <- c()
rangeStat_US <- c()
rangePvalue_US <- c()

# Super-Duper

In [ ]:
seedvalues <- c(100, 233424280)

for(nn in n){
  for(kk in k){
    if(kk >= nn){
      next
    }
    
    for(ss in seedvalues){
      
      set.seed(ss, kind = "Super-Duper")
      edistr <- getEmpiricalDistrRaw(nn, kk, reps)
      itemFreq <- getItemFreq(getItemCounts(edistr), reps)
      samFreq <- getEmpiricalDistr(edistr)
      
      maxProb <- c(maxProb, max(itemFreq$Probability))
      minProb <- c(minProb, min(itemFreq$Probability))
      meanProb <- c(meanProb, mean(itemFreq$Probability))
      maxProbRatio <- c(maxProbRatio, computeMaxProbRatio(itemFreq$Probability))
      nvalues <- c(nvalues, nn)
      kvalues <- c(kvalues, kk)
      prng <- c(prng, "Super-Duper")
      seed <- c(seed, ss) 
      
      # First order
      chisqtest <- conductChisqTest(itemFreq$Count)
      chisqDF_FO <- c(chisqDF_FO, chisqtest$DF)
      chisqStatistic_FO <- c(chisqStatistic_FO, chisqtest$Statistic)
      chisqPvalue_FO <- c(chisqPvalue_FO, chisqtest$Pvalue)
        
      rangeObserved_FO <- diff(range(itemFreq$Count))
      rangeStat_FO <- c(rangeStat_FO, rangeObserved_FO)
      rangePvalue_FO <- c(rangePvalue_FO, 1-distrMultinomialRange(rangeObserved_FO, reps*kk, nn))
        
      # Unique samples
      sampleFreqVec <- sapply(samFreq, function(x) x$freq)
      chisqtest <- conductChisqTest(sampleFreqVec)
      chisqDF_US <- c(chisqDF_US, chisqtest$DF)
      chisqStatistic_US <- c(chisqStatistic_US, chisqtest$Statistic)
      chisqPvalue_US <- c(chisqPvalue_US, chisqtest$Pvalue)
      
      rangeObserved_US <- diff(range(sampleFreqVec))
      rangeStat_US <- c(rangeStat_US, rangeObserved_US)
      rangePvalue_US <- c(rangePvalue_US, 1-distrMultinomialRange(rangeObserved_US, reps, choose(nn, kk)))
    }
  }
}

In [ ]:
seedvalues <- c(100, 233424280, 429496729)

for(nn in n){
  for(kk in k){
    if(kk >= nn){
      next
    }
    
    for(ss in seedvalues){
      
      set.seed(ss, kind = "Mersenne-Twister")
      edistr <- getEmpiricalDistrRaw(nn, kk, reps)
      itemFreq <- getItemFreq(getItemCounts(edistr), reps)
      samFreq <- getEmpiricalDistr(edistr)

      maxProb <- c(maxProb, max(itemFreq$Probability))
      minProb <- c(minProb, min(itemFreq$Probability))
      meanProb <- c(meanProb, mean(itemFreq$Probability))
      maxProbRatio <- c(maxProbRatio, computeMaxProbRatio(itemFreq$Probability))
      nvalues <- c(nvalues, nn)
      kvalues <- c(kvalues, kk)
      prng <- c(prng, "Mersenne Twister")
      seed <- c(seed, ss)
      
      # First order
      chisqtest <- conductChisqTest(itemFreq$Count)
      chisqDF_FO <- c(chisqDF_FO, chisqtest$DF)
      chisqStatistic_FO <- c(chisqStatistic_FO, chisqtest$Statistic)
      chisqPvalue_FO <- c(chisqPvalue_FO, chisqtest$Pvalue)
        
      rangeObserved_FO <- diff(range(itemFreq$Count))
      rangeStat_FO <- c(rangeStat_FO, rangeObserved_FO)
      rangePvalue_FO <- c(rangePvalue_FO, 1-distrMultinomialRange(rangeObserved_FO, reps*kk, nn))
        
      # Unique samples
      sampleFreqVec <- sapply(samFreq, function(x) x$freq)
      chisqtest <- conductChisqTest(sampleFreqVec)
      chisqDF_US <- c(chisqDF_US, chisqtest$DF)
      chisqStatistic_US <- c(chisqStatistic_US, chisqtest$Statistic)
      chisqPvalue_US <- c(chisqPvalue_US, chisqtest$Pvalue)
      
      rangeObserved_US <- diff(range(sampleFreqVec))
      rangeStat_US <- c(rangeStat_US, rangeObserved_US)
      rangePvalue_US <- c(rangePvalue_US, 1-distrMultinomialRange(rangeObserved_US, reps, choose(nn, kk)))
    }
  }
}

# First-order selection probabilities, summary statistics


In [ ]:
d1 <- cbind(nvalues, kvalues, prng, seed, minProb, meanProb, maxProb, maxProbRatio)
rownames(d1) <- NULL

ord <- order(nvalues, kvalues, prng, seed)
d1[ord, ]

# First order selection probabilities, chi-squared test and range statistic

We first test whether each item $1, \dots, k$ is selected with equal probability. We do two tests: the usual chi-squared test and another test based on the range of the multinomial values, $max_i n_i - min_i n_i$, where $n_1, \dots, n_k$ are the number of items in each of $k$ cells that have equal probability $1/k$.

Johnson and Young (1960) and Young (1962) provide the following approximation to the distribution of the range

$$P(\max_i n_k - \min_i n_k \leq r) \approx P(W_m \leq (r-(2n)^{-1})(m/n)^{1/2})$$

where $W_m$ denotes the sample range of $m$ independent standard normal random variables. It is a known result (see e.g. Pearson and Hartley p. 43, 1954 or Ruben, 1960) that the distribution function for the range of IID normal samples is given by

$$R(w) = n \int_{-\infty}^{\infty} \phi(x)\left[ \Phi(x+w) - \Phi(x)\right]^{n-1}dx$$

where $\phi$ and $\Phi$ are the standard normal density and cumulative distribution function, respectively.  We leverage these two results to approximate the p-value of the range statistic.

In [ ]:
d2 <- cbind(nvalues, kvalues, prng, seed, chisqStatistic_FO, chisqDF_FO, chisqPvalue_FO, rangeStat_FO, rangePvalue_FO)
rownames(d2) <- NULL
d2[ord, ]

# Selection probabilities for unique samples, chi-squared test

In [ ]:
d3 <- cbind(nvalues, kvalues, prng, seed, chisqStatistic_US, chisqDF_US, chisqPvalue_US, rangeStat_US, rangePvalue_US)
rownames(d3) <- NULL
d3[ord, ]